# Sentiment Analysis -NLP Project 1

## 1. Data loading

In [15]:
import nltk
from datasets import load_dataset
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

dataset = load_dataset("imdb")
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    tokens = word_tokenize(text)
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

sample_text = dataset["train"]["text"][0]
processed_text = preprocess_text(sample_text)
print("Original:", sample_text[:100])
print("Processed:", processed_text[:100])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sandeepsony/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sandeepsony/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/sandeepsony/nltk_data'
    - '/Users/sandeepsony/Projects/PandasPrep/.venv/nltk_data'
    - '/Users/sandeepsony/Projects/PandasPrep/.venv/share/nltk_data'
    - '/Users/sandeepsony/Projects/PandasPrep/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
